In [3]:
import cv2
import os
import time
import uuid

In [6]:
IMAGE_PATH="Tensorflow/workspace/images/collectedimages"

In [8]:
labels=['hello','thanks','yes','no','iloveyou']
number_imgs=15

In [13]:
for label in labels:
    !mkdir { 'Tensorflow\workspace\images\collectedimages\\' +label }
    cap=cv2.VideoCapture(0)
    print("Collecting for images {}".format(label))
    time.sleep(5)
    for imgnum in range(number_imgs):
        ret, frame=cap.read()
        imagename=os.path.join(IMAGE_PATH,label,label+"."+'{}.jpg'.format(str(uuid.uuid1())))
        cv2.imwrite(imagename,frame)
        cv2.imshow('frame',frame)
        time.sleep(2)
        
        if cv2.waitKey(1) & 0xff == ord('q'):
            break
    
    cap.release()

SyntaxError: expected ':' (1713977855.py, line 6)